### Задание 1

Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.

In [47]:
import pandas as pd

In [48]:
ratings = pd.read_csv('ml-latest-small/ratings.csv')
ratings.head()

userId  movieId  rating   timestamp
0       1       31     2.5  1260759144
1       1     1029     3.0  1260759179
2       1     1061     3.0  1260759182
3       1     1129     2.0  1260759185
4       1     1172     4.0  1260759205

In [49]:
rating2 = ratings.groupby('userId').agg({'timestamp': ['min', 'max'], 'movieId': 'count'}).reset_index()

In [50]:
(rating2[rating2['movieId']['count'] > 100]['timestamp']['max'] - rating2[rating2['movieId']['count'] > 100]['timestamp']['min']).mean()

40080507.4496124

### Задание 2

Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия). Необходимо сформировать две таблицы:
таблицу с тремя типами выручки для каждого client_id без указания адреса клиента  
аналогичную таблицу по типам выручки с указанием адреса клиента  
Обратите внимание, что в процессе объединения таблиц данные не должны теряться.  

In [52]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

client_id  rzd_revenue
0        111         1093
1        112         2810
2        113        10283
3        114         5774
4        115          981

In [53]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

client_id  auto_revenue
0        113         57483
1        114            83
2        115           912
3        116          4834
4        117            98

In [54]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

client_id  air_revenue
0        115           81
1        116            4
2        117           13
3        118          173

In [55]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base

client_id          address
0        111  Комсомольская 4
1        112   Энтузиастов 8а
2        113  Левобережная 1а
3        114          Мира 14
4        115        ЗЖБИиДК 1
5        116    Строителей 18
6        117  Панфиловская 33
7        118     Мастеркова 4

In [96]:
# таблица 1
((pd.merge(pd.merge(pd.merge(client_base.loc[:,'client_id'], rzd, how = 'left', on = 'client_id'), 
                    auto, how = 'left', on = 'client_id'), 
                       air, how = 'left', on = 'client_id'))
 .fillna(0))

client_id  rzd_revenue  auto_revenue  air_revenue
0        111       1093.0           0.0          0.0
1        112       2810.0           0.0          0.0
2        113      10283.0       57483.0          0.0
3        114       5774.0          83.0          0.0
4        115        981.0         912.0         81.0
5        116          0.0        4834.0          4.0
6        117          0.0          98.0         13.0
7        118          0.0           0.0        173.0

In [63]:
# таблица 2
(client_base
 .merge(rzd, how = 'left', on = 'client_id')
 .merge(auto, how = 'left', on = 'client_id')
 .merge(air, how = 'left', on = 'client_id')
 .fillna(0))

client_id          address  rzd_revenue  auto_revenue  air_revenue
0        111  Комсомольская 4       1093.0           0.0          0.0
1        112   Энтузиастов 8а       2810.0           0.0          0.0
2        113  Левобережная 1а      10283.0       57483.0          0.0
3        114          Мира 14       5774.0          83.0          0.0
4        115        ЗЖБИиДК 1        981.0         912.0         81.0
5        116    Строителей 18          0.0        4834.0          4.0
6        117  Панфиловская 33          0.0          98.0         13.0
7        118     Мастеркова 4          0.0           0.0        173.0

### Задание 3

В задаче сквозной аналитики вам предоставили данные по местоположению пользователей. Т. е. для каждого user_id известна последовательность координат (широта/долгота), когда они требовались приложению для полноценной работы. Как бы вы добавили эти сведения в таблицу визитов и покупок? Для составления ответа можно использовать вопросы:
* У каждого пользователя известен набор координат. А для связывания с визитом или фактом покупки скорее всего потребуется одно-два числа. Как их получить?
* Наборы координат одного и того же пользователя могут быть значительно удалены друг от друга. Как это отразится на вопросе расчетах пункта 1?
* Какие дополнительные признаки можно получить из координат? Ведь это просто числа, которые сами по себе мало что дают.

* У каждого пользователя известен набор координат. А для связывания с визитом или фактом покупки скорее всего потребуется одно-два числа. Как их получить?  

Основным ключем таблицы с координатами для связывания должен стать id события, который необходимо добавить в соответствующие колонки таблиц визитов и покупок

* Наборы координат одного и того же пользователя могут быть значительно удалены друг от друга. Как это отразится на вопросе расчетах пункта 1?  

при использовании уникального id события устанавливается единое местоположение каждого события, дублирование не происходит.

* Какие дополнительные признаки можно получить из координат? Ведь это просто числа, которые сами по себе мало что дают.

установить область максимального количества заказов для определения оптимальной геолокации склада, можно использовать для построения оптимальных логистических маршрутов доставки.